# Attention Mechanism 

- Seq2Seq의 입력 Sequence를 Encoder에 의해 하나의 Context Vector로 변환하고, Decoder가 이 Vector를 이용해 출력 Sequence 구성
- 단점 :
  - 1. Context Vector의 고정된 크기(Columns -> Matrix의 Row x Col)로 인해 정보 손실
  - 2. 긴 Sequence의 경우, RNN 계열 모델들의 장기기억소실/기울기소실 문제들이 발생

- Attention Mechanism : Decoder에서 출력 단어를 생성할 때, 단어를 예측하느 매 시점(Time Step)마다 Encoder의 전체 입력문장의 정보를 반영하여 출력
- 해당 시점에서 예측해야할 단어와 연관있는 입력단어의 부분을 좀 더 집중(Attention)하여 출력
- 작동 :
  - 모든 단어 쌍에 대한 key Value 형태의 유사도 분석을 수행
  - 해당 유사도를 이용하여 Attention Score 계산
  - Attention Score : 현재 Decoder의 특정 t 시점에서 단어를 예측하기 위해, Encoder의 각 시점의 값이 Decoder의 각 시점의 값과 얼마나 유사한지를 계산
  - Attention Score를 이용해 분포추정 실시 -> 출력 단계엣 좀 더 유사한 단어 쌍이 생성되도록 유도

![image1](https://i0.wp.com/blog.kakaocdn.net/dn/RtMGM/btrXyiWoa10/5zt5t6BgqEXrtsU13OpHdK/img.png?w=900&ssl=1)

# 트랜스포머 (Transformer)

- Attention Mechanism을 응용하여, 이전의 RNN, LSTM모델에서 Sequence 단위의 데이터를 순차적으로 처리하는 것과 달리, 전체 Sequence를 한번에 처리
- **구성 요소**
  - Self - Attention : 하나의 Sequence 내 각 위치의 토큰이 Sequence내 다른 위치 토큰과 얼마나 관련이 있는지를 계산
      - 문장 내에 단어간의 등장 순서/ 관계 파악

  - Multi-Head Attention : Self - Attention 층이 병렬로 반복적으로 처리되며 다양한 표현(Subspace)에서 정보를 추출
      - 위치정보가 처리된 문장의 정보를 처리
         
  - Position Encoding : Attetion으로 구성된 모델에서 각 Sequence의 순서 정보를 유지하기 위해, 순서 정보를 가진 Position Encoding을 수행
 
![image1](https://wikidocs.net/images/page/31379/transformer_attention_overview.PNG)

In [1]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ------ --------------------------------- 0.8/5.1 MB 26.4 MB/s eta 0:00:01
   ----------------- ---------------------- 2.3/5.1 MB 29.1 MB/s eta 0:00:01
   ----------------------------- ---------- 3.7/5.1 MB 33.6 MB/s eta 0:00:01
   ---------------------------------------  5.1/5.1 MB 35.9 MB/s eta 0:00:01
   ---------------------------------------- 5.1/5.1 MB 32.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/161.5 kB ? eta -:--:--
   ---------------------------------------- 161.5/161.5 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.3 kB ? eta -:--:--
   ---------------------------------------- 101.3/101.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/113.6 kB ? eta -:--:--
   ---------------------------------------- 113.6/113.6 kB ? eta 0:00:00
   --

In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow_datasets as tfds
import tensorflow as tf

In [7]:
path = 'C:/Users/UserK/Desktop/Ranee/data/ML/'

In [8]:
df1 = pd.read_csv(path+'39_Data.csv')

In [9]:
# 특수 문자에 대해 띄어쓰기를 수행 -> 토큰화 
# ?!., 기호 앞뒤로 공백을 추가 -> 특수기호에 대한 공백 추가 후, strip 함수를 이용해 문장 앞뒤로 공백을 제거 
questions = [re.sub(r"([?.!,])", r" \1 ", x).strip() for x in df1['Q']]
answers   = [re.sub(r"([?.!,])", r" \1 ", x).strip() for x in df1['A']]

In [11]:
# 서브토크나이저 : 훈련 데이터에 없는 새로운 단어가 등장해도 그 단어를 구성하는 문자를 분해하여(서브 워드) 처리 할 수 있음 
token_model = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [16]:
# tfds 라이브러리 오류 
import pickle
token_model = pickle.load(open('token.sav', 'rb'))

In [17]:
pickle.dump(token_model, open('token.sav', 'wb'))

In [45]:
# 처리된 토큰 정보 
# token_model.subwords

In [18]:
# 만들어진 토큰 수 
token_model.vocab_size

8178

In [19]:
# SOS 와 EOS 를 단어 사전에 추가 
# SOS 와 EOS 가 들어갈 Index 부여 
START_TOKEN, END_TOKEN = [token_model.vocab_size] , [token_model.vocab_size + 1] 

# SOS과 EOS 을 추가하기 위해 단어 사전의 공간을 늘림 
VOCAB_SIZE = token_model.vocab_size + 2 

In [20]:
# 앞서 구성한 서브토크나이저를 이용해 Text to Sequence 
print(questions[0])
token_model.encode(questions[0])

12시 땡 !


[7915, 4207, 3060, 41]

In [21]:
# 숫자를 문자로 변환 가능 
seq1 = [5766,611,3509,141,685,3747,849]
token_model.decode(seq1)

'가스비 비싼데 감기 걸리겠어'

- 전체 데이터를 이용한 Text to Sequence와 Padding 작업을 수행 

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
# 토큰화 + 패딩 
def token_pad(text):
    text_list = [ ] # 토큰화 + 패딩 결과를 담을 변수를 선언 
    for i in text :
        sent1  = START_TOKEN + token_model.encode(i) + END_TOKEN # 문장을 Encoding 후, SOS 번호와 EOS 번호를 추가
        text_list.append(sent1)
    return pad_sequences(text_list, maxlen=40, padding='post')

In [24]:
questions_out = token_pad(questions)
answers_out   = token_pad(answers)

- 앞서 처리된 질문/답변 행렬을 데이터 파일로 지정

In [25]:
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE

- 교사 강요 (Teacher Forcing) : 모델의 현재 출력을 다음 시점의 입력으로 사용하는 대신, 실제 목표 분장의 현재 시점의 데이터를 다음 시점의 입력으로 사용
  - "나는 고양이를 정말 좋아해" 문장에서 "고양이" 단어가 나올 때, "정말"이라는 단어가 등장하게 학습이 되어야 하지만, "강아지"와 같은 전혀 다른 단어들이 출현 
  - 학습 단계에서 이전 단어 다음의 단어를 정확하게 맞추지 못해도, 다음 단어의 입력을 강제로 정답을 알려주며 학습을 수행 

In [26]:
# Tensor flow의 Dataset를 이용해 배치사이즈로 데이터를 묶음 
batch_size = 64 

dataset = Dataset.from_tensor_slices( (
    # 디코더의 입력 / 마지막 패딩 토큰을 제거 
    {'inputs' : questions_out, 'dec_inputs' : answers_out[:, :-1]},
    # 디코더의 출력에서 EOS 를 제거 
    {'outputs' : answers_out[:, 1:]}
) )

# 데이터셋을 빠르게 로드하여 Epoch 수행 
dataset1 = dataset.cache()
# 데이터를 Batch Size에 맞게 묶음 
dataset2 = dataset1.batch(batch_size)
# 데이터의 로딩 시간을 줄이기 위해, 데이터를 미리 메모리에 로딩 
dataset3 = dataset2.prefetch(AUTOTUNE)

In [27]:
save_path = '/chatbot_dataset'
dataset3.save(save_path)

# 포지셔널 인코딩 (Positional Encoding)

- Transformer 모델의 입력층으로 사용되는 Layer
- 기본적인 RNN 모델은 단어를 순차적으로 받아서 처리하기 때문에, 문장 내 단어의 순서가 처리되는 정보가 존재
- 그러나 Transformer 모델은 데이터를 한번에 처리하기 때문에, 문장 내 단어의 순서 정보가 소실될 수 있음
- ![image1](https://wikidocs.net/images/page/31379/transformer2.PNG)

- Positional Encoding 기법으로 단어의 위치정보를 Matrix로 형태로 변환하여 학습을 수행
- sin 함수와 cos 함수를 활용하여, 단어의 순서 정보를 Matrix 형태로 단어 Matrix와 함께 전달

- ![image1](https://wikidocs.net/images/page/31379/transformer6_final.PNG)


# Transformer Attention 

- Attention Mechanism : 특정 Layer층에서 출력되는 매 시점 마다, 전체 문장을 다시 참조하여 출력 
- 유사도를 계산하여, 가장 유사도가 높은 문장의 단어 벡터를 찾아 다음 Layer에 반영
- 어텐션 메커니즘의 핵심 개념:
    - 쿼리(Query): 현재 처리 중인 단어(또는 토큰)에 대한 표현
    - 키(Key): 비교 대상이 되는 단어들의 표현
    - 밸류(Value): 각 키와 쿼리 얻어진 유사도 값 

- Transformer Model에서는 3파트의 Attention이 존재
  - Multi-Head Attention (Encoder-Decoder Attention) : Encoder와 Decoder를 이어주는 Attention / Head를 통해 병렬로 처리
  - Encoder Self Attention: 모델이 하나의 Seq 내에서 단어들 사이의 관계를 파악
  - Masked Decoder Self Attention : 모델이 하나의 Seq 내에서 단어들 사이의 관계를 파악
 
  - ![image2](https://wikidocs.net/images/page/31379/transformer_attention_overview.PNG)

In [32]:
import transformer_chatbot

In [33]:
tf.keras.backend.clear_session() 

# 하이퍼 파라미터 세팅 
d_model = 256 # 트랜스 포머 내 모든 레이어의 Node 수 
num_layer = 2 # 인코더와 디코더에 각각 있는 Layer 수 
num_heads = 8 # 멀티-헤드 어텐션에서 사용되는 Head 수 
dff = 512 # Feed Forward 신경망 내 Layer Node 수 
drop_out = 0.1 # 10%의 랜덤 비율로 Layer Node를 비활성화 

model = transformer_chatbot.transformer(
    vocab_size= VOCAB_SIZE, # 단어 사전 수 
    num_layers= num_layer,
    dff = dff, d_model= d_model, num_heads=num_heads, dropout=drop_out)

In [34]:
# 모델 컴파일을 위한 정확도 계산 함수를 구성 
# 시퀀스 최대 길이에서 시작토큰과 종료토큰을 제외하고 나머지 데이터를 이용해 정확도를 계산 
def accuracy(y_true, y_pred):
    # Label (Decoder Input) Size : Batch size x Max length - 1 
    y_true1 = tf.reshape(y_true, shape=(-1, 40 - 1 ))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true1, y_pred)

In [35]:
# 적용 할 Learning Ratet를 각 Node에 따라 가변으로 (동적으로) 조정 
learning_rate = transformer_chatbot.CustomSchedule(d_model)
# 최적화 함수 설정 / 각 노드 별로 학습율도 조절하면서, 모멘텀 벡터도 계산 
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [36]:
model.compile(optimizer=optimizer, loss=transformer_chatbot.loss_function, metrics=[accuracy])

In [37]:
model.fit(dataset3, epochs=20) # 100회이상 학습시 적절한 성능이 나옴 

Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 83s 392ms/step - accuracy: 0.0179 - loss: 1.4186
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 71s 382ms/step - accuracy: 0.0485 - loss: 1.1541
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 75s 408ms/step - accuracy: 0.0495 - loss: 0.9602
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 75s 406ms/step - accuracy: 0.0518 - loss: 0.8811
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 73s 395ms/step - accuracy: 0.0549 - loss: 0.8257
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 73s 396ms/step - accuracy: 0.0576 - loss: 0.7744
Epoch 7/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 73s 397ms/step - accuracy: 0.0613 - loss: 0.7201
Epoch 8/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 74s 398ms/step - accuracy: 0.0657 - loss: 0.6657
Epoch 9/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 72s 388ms/step - accuracy: 0.0720 - loss: 0.6095
Epoch 10/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 71s 385ms/step - accuracy: 0.0801 - loss: 0.5431
Epoch 11/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 74s 397ms/step - accuracy: 0.0878 - loss: 0.4791
Epoch 12/20
185/185

In [38]:
# 입력값을 처리하는 함수 
def preprocess_sentence(sentence):
    # 입력 문장에 대한 특수 기호 처리
    sent1 = re.sub(r"([?.!,])", r" \1 ", sentence).strip()
    return sent1

In [39]:
def evaluate(sentence) :
    # 입력문장에 대한 전처리
    sent1 = preprocess_sentence(sentence)
    # 입력문장에 시작토큰과 종료 토큰을 추가
    sent2 = tf.expand_dims( START_TOKEN + token_model.encode(snet1) _ END_TOKEN, axis=0)
    output = tf.expand_dims( START_TOKEN , 0)

    # 디코더에 의한 예측값 시작
    for i in range(40) :
        predictions = model(input= [sent2, output], trainig = False)

        # 모델이 출력에서 마지막 단어를 선택해 이를 바탕으로 다음 단어를 순차적으로 예측
        prediconts = predictions[:, -1:, :]
        # 예측된 단어를 변수로 선언
        pred_id = tf.cast(tf.argmax(prediconts, axis=-1), tf.int32)
        # 만약 종료 토큰이 등장하면 예측을 중단
        if tf.equal(pred_id, END_TOKEN[0]) :
            break
        # 현재 시점의 예측단어를 Output에 연결
        output = tf.concat([output, pred_id], axis=-1)

    return tf.squeeze(output, axis=0)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (202853579.py, line 5)

In [ ]:
# 문자로 Decoding
def predict(sent1) :
    pred1 = evaluate(sent1)
    pred_sent = token_model.decode( [x for x in pred1 if x < token_model.vocab_size] )
    print("입력문장 : " , sent1)
    print("답변문장 : ", pred_sent)
    